In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


import string

import re

import json

In [2]:
def mergeDictionary(dict_1, dict_2):
    dict_3 = {**dict_1, **dict_2}
    for key, value in dict_3.items():
        if key in dict_1 and key in dict_2:
            dict_3[key] = value + dict_1[key]
    return dict_3

In [3]:
import nltk
nltk.download('words')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package words to /home/cdsw/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
words = set(nltk.corpus.words.words())

In [6]:
def embed(input):
    return model(input)

In [7]:
def basic_stats_error(datalist):
    
    if len(datalist)>1:
        result_describe = list(describe(datalist)[2:])
        result_percentile = result_describe + list(np.percentile(datalist,[10,25,50,75,90]))
        array = np.array(datalist)
        result_percentile = result_percentile+[len(array),len(array[np.where(array>0)[0]])/len(array)*100,
                                               len(array[np.where(array<0)[0]])/len(array)*100]
    else:
        result_percentile = []

    return result_percentile

In [8]:
def df_lambda_detect_not_words(df, column_name, new_column):
    Lem = WordNetLemmatizer()
    df[new_column] = df[column_name].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if Lem.lemmatize(w.lower()) not in words))
    return df

In [9]:
def df_lambda_remove_punctuation(df, column_name, new_column):
    df[new_column] = df[column_name].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return df

In [10]:
def df_lambda_replace_characters(df, column_name, new_column, dictionary):
    df[new_column] = df[column_name].apply(lambda x: replace_words(x, dictionary))       
    return df


In [11]:
def replace_words(strtext, dictionary):
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], strtext)
    return result

In [12]:
with open("replacement_dictionary_ordered.json") as json_file:
    replacement_dictionary_2 = json.load(json_file)


In [13]:
import glob
nounsentence_files = glob.glob("comments/raw_comments/*_filtered_noun_sentences.fea")

In [14]:
nounsentence_files[42:43]

['comments/raw_comments/agent_comments_TX_filtered_noun_sentences.fea']

In [ ]:
for file in nounsentence_files[42:43]:
    
    print(file)
    noun_sentence_df = pd.read_feather(file)
    print(len(noun_sentence_df))
    tic = time.time()
    
    noun_sentence_df_20 = noun_sentence_df[noun_sentence_df['count']>20]
    
    noun_sentence_df_20 = df_lambda_remove_punctuation(noun_sentence_df_20, 'noun_sentence', 'noun_sentence_no_punct')
    
    noun_sentence_df_20 = df_lambda_replace_characters(noun_sentence_df_20,  'noun_sentence_no_punct', 
                                                   'noun_sentence_corrections', replacement_dictionary_2)
    
    import time
    columns = ['noun_sentence_corrections']
    for column in columns:
        print(column)
        tic = time.time()
        new_column = column + '_embeddings'
        noun_sentence_df_20[new_column] = noun_sentence_df_20[column].apply(lambda x: embed([x]).numpy()[0]) 
        toc = time.time()
        print(toc - tic)
        
    noun_sentence_df_1000 = noun_sentence_df_20[noun_sentence_df_20['count']>1000]
    noun_sentence_df_1000_corr = noun_sentence_df_1000[['noun_sentence_corrections',
                                                        'noun_sentence_corrections_embeddings']]
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()
    
    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)
    
    noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr, split_df], axis=1)
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column])
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns=['index'])
    
    correlations_df = noun_sentence_df_1000_corr.corr(method='pearson')
    
    skill_group = {}
    test_df = correlations_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        #print(noun_sentence_df_20_reindex.loc[group,'AVM_Error_Average'])
        skill_group[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)

    noun_sentence_df_50_1000 = noun_sentence_df_20[(noun_sentence_df_20['count']>50)
                                                   &(noun_sentence_df_20['count']<=1000)]
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.loc[:,['noun_sentence_corrections_embeddings']]
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.reset_index()

    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_50_1000_reindex[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_50_1000_reindex[column].tolist(), columns=column_names)
    
    noun_sentence_df_50_1000_corr = pd.concat([noun_sentence_df_50_1000_reindex, split_df], axis=1)
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.drop(columns = [column])
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.T
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.reset_index()
    
    noun_sentence_df_1000_reindex = noun_sentence_df_1000.set_index('noun_sentence_corrections')
    
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_1000_reindex_df = pd.DataFrame(columns=columns)
    
    for key in skill_group:
        #print(key)
        #print(skill_group[key])
        if len(skill_group[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(\
                                                        noun_sentence_df_1000_reindex.loc[key,columns])

        else:
            print(skill_group[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_1000_reindex.loc[skill_group[key],columns].iloc[0])
            for ii in range(3):
                temp_df[noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]] = \
                                           [np.concatenate(noun_sentence_df_1000_reindex.loc[skill_group[key],columns]\
                                               [noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]].values)]
            noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(temp_df.loc[:,columns])
            subgroup = skill_group[key][1:]
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_reindex_df[['noun_sentence_corrections_embeddings']]
    
    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)
    
    noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr.reset_index(), split_df], axis=1)
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column]).set_index('index')

    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index().drop(columns=['index'])
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.rename(columns={'noun_sentence_corrections':'index'})
    
    noun_sentence_df_50_1000_corr.index.rename('index')
    
    correlations_50_1000_df = noun_sentence_df_50_1000_corr.corr()
    
    skill_group_50_1000 = {}
    test_df = correlations_50_1000_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        print(len(test_df))
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        skill_group_50_1000[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)
        
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')
    
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_50_1000_reindex_df = pd.DataFrame(columns=columns)
    
    
    for key in skill_group_50_1000:
        print(key)
        #print(skill_group[key])
        if len(skill_group_50_1000[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(\
                                                        noun_sentence_df_50_1000_reindex.loc[key,columns])

        else:
            #print(skill_group_50_1000[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_50_1000_reindex.loc[[skill_group_50_1000[key][0]],columns].iloc[[0]])
            for ii in range(3):
                temp_df[noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]] = \
                                           [np.concatenate(noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns]\
                                               [noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]].values)]
            noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(temp_df.loc[:,columns])
            subgroup = skill_group_50_1000[key][1:]
            
    noun_sentence_df_full_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(noun_sentence_df_50_1000_reindex_df)
        
    skill_group_full = mergeDictionary(skill_group, skill_group_50_1000)

    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_reindex_df[['noun_sentence_corrections_embeddings']]

    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_full_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_full_1000_corr[column].tolist(), columns=column_names)

    noun_sentence_df_full_1000_corr = pd.concat([noun_sentence_df_full_1000_corr.reset_index(), split_df], axis=1)
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.drop(columns = [column]).set_index('index')
    
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.T
    
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.reset_index().drop(columns=['index'])
    
    correlations_full_1000_df = noun_sentence_df_full_1000_corr.corr()
    
    skill_group_full_1000 = {}
    test_df = correlations_full_1000_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        print(len(test_df))
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        skill_group_full_1000[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)
        
    skill_group_full_final = skill_group_full_1000.copy()

    for key in skill_group_full_1000:
        list_tmp = skill_group_full[key]
        skill_group_full_final[key] = skill_group_full_1000[key] + list_tmp
        
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_final_reindex_df = pd.DataFrame(columns=columns)
    
    
    for key in skill_group_full_1000:
        print(key)
        #print(skill_group[key])
        if len(skill_group_full_1000[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(\
                                                        noun_sentence_df_full_1000_reindex_df.loc[key])

        else:
            #print(skill_group_50_1000[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_full_1000_reindex_df.loc[[skill_group_full_1000[key][0]]].iloc[[0]])
            for ii in range(3):
                temp_df[noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]] = \
                                   [np.concatenate(noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]]\
                                   [noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]].values)]
            noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(temp_df)
            
    noun_file = file.split('.')[0] + '_noun_sentences_df_final_reindex.fea'
    noun_sentence_df_final_reindex_df.reset_index().to_feather(noun_file)
    
    import json
    # create json object from dictionary
    json = json.dumps(skill_group_full_final, indent = 4)
    f = open(noun_file+"noun_phrases_full_final.json","w")
    f.write(json)
    f.close()


comments/raw_comments/agent_comments_TX_filtered_noun_sentences.fea
5650881


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


noun_sentence_corrections


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


346.1794376373291
new bathrooms
1
quartz counters
3
new insulation
2
top bottom
1
barn door
1
new cabinetry
1
updates list
1
new cabinets
2
new doors
1
high end finishes
1
new vanities
1
foam insulation
2
new windows
1
outdoor shower
1
new kitchen
3
clean lines
1
marble countertops
4
primary bath
1
new driveway
1
pot filler
1
primary suite
1
subway tile
2
custom kitchen
2
new double pane windows
2
exterior doors
1
new energy
1
complete remodel
1
new stainless steel appliances
2
attention detail
1
new construction home
2
fire pit
1
outdoor kitchen
1
smart home
1
high end
1
new vanity
1
upgrades list
1
new exterior
1
pella windows
1
new hvac
2
electrical panel
1
efficient windows
1
wood look tile
3
exotic granite
1
new deck
1
modern home
1
designer tile
1
boat dock
1
wine fridge
2
custom closet
1
new sinks
1
entertainers dream
1
new appliances
1
new tub
1
frameless glass shower
2
ice maker
1
new fixtures
1
new floors
2
new hardware
1
custom cabinetry
3
salt water pool
2
country home
1
in

...

['new granite counter tops', 'new granite', 'new granite countertops', 'granite counter', 'new granite counters', 'granite countertops', 'granite countertop', 'granite counter tops', 'granite countertops', 'kitchen granite counter tops', 'beautiful granite counter tops', 'granite countertops', 'granite counter top', 'granite countertops', 'granite kitchen counter tops']
['travertine tile', 'travertine backsplash', 'tile backsplash', 'tile backsplash']
['concrete floor', 'concrete floors', 'concrete slab']
['amazing backyard', 'gorgeous backyard', 'beautiful backyard', 'wonderful backyard', 'huge backyard', 'great backyard', 'lovely backyard', 'nice backyard']
['new hardwood floors', 'hard wood floors', 'hardwood floors', 'hardwood floors']
['burner gas range', 'burner gas stove', 'burner gas cooktop', 'burner gas cook top']
['white cabinetry', 'white cabinets']
['new tile', 'new tile floors']
['roomy kitchen', 'spacious kitchen']
['new garage doors', 'garage doors', 'new garage door']


['acre lot', 'acre land', 'acre corner lot']
['grand foyer', 'large fireplace']
['private baths', 'private bath']
['bay window master', 'master bay window']
['ample closet space', 'great closet space', 'plenty closet space', 'ample cabinet space', 'lots closet space']
['double oven', 'double oven']
['recent ac', 'recent ac']
['oak trees', 'large oak trees']
['storm doors', 'storm door']
['gas fireplace', 'gas fireplace']
['water softener system', 'water softener']
['huge culdesac lot', 'culdesac lot', 'culdesac lot', 'large culdesac lot']
['extensive crown', 'lots crown']
['ceramic tile kitchen', 'ceramic tile entry']
['ac unit', 'ac unit']
['awesome home', 'nice home']
['master w', 'master w']
['fans rooms', 'fans bedrooms']
['katy isd schools', 'katy isd', 'katy schools']
['gourmet kitchen granite counters', 'kitchen with granite counters', 'island kitchen granite counters']
['ample storage', 'large storage', 'great storage', 'ample storage space', 'lots storage', 'good storage']
['s

['master suite walkin closet', 'master bedroom walkin closet']
['wonderful place', 'wonderful location']
['entire family', 'whole family']
[' ', '  ', '   ']
['friendly neighbors', 'great neighbors']
['great closets', 'huge closets']
['spacious game room', 'large game room']
['plenty space', 'lots space']
['major highways', 'major freeways', 'major roads']
['downstairs master bedroom', 'downstairs bedroom', 'master bedroom downstairs']
['great open floorplan', 'wonderful floorplan', 'great floorplan']
['upstairs master', 'downstairs master']
['art niche', 'art niches']
['black appliances', 'white appliances']
['breakfast room', 'breakfast room']
['quiet culdesac street', 'quiet culdesac lot', 'quiet street']
['tile floors entry', 'tile entry']
['private study french doors', 'study french doors']
['plenty room', 'lots room']
['distance schools', 'distance school']
['flexible floorplan', 'flexible floor plan']
['tons counter space', 'lots counter space', 'tons space']
['backyard neighbor

steam oven
30925
1
ge stainless steel appliances
30924
4
restoration hardware
30920
1
optional garden tub
30919
1
stall barn
30918
2
timeless style
30916
1
property cross
30915
1
warm inviting kitchen
30914
2
stock tank
30912
1
kitchen boasts quartz countertops
30911
7
top line finishes
30904
1
marvin windows
30903
1
beautiful quartz countertops
30902
9
redone top bottom
30893
1
new wood cabinets
30892
2
new wood vinyl plank
30890
8
modern bathrooms
30882
3
new gourmet kitchen
30879
6
vanities fixtures
30873
1
exterior cameras
30872
2
new pex
30870
1
new electrical system
30869
1
extreme energy efficiency w16 seer hvac
30868
2
main body lake
30866
3
refrigerator mini blinds
30863
1
perfect country
30862
2
seam roof
30860
2
outdoor kitchen pergola
30858
4
new pantry
30854
1
ready march
30853
1
amenity list
30852
3
pedestal tub
30849
8
large barn
30841
2
white quartz counters
30839
3
gorgeous acres
30836
3
guest cottage
30833
1
road frontage
30832
1
amazing views lake
30831
12
new finish

claw tub
29877
2
deep water
29875
1
furniture vanities master bath
29874
2
kitchen optional island
29872
1
modern flair
29871
1
impeccable finishes
29870
6
travertine patio
29864
34
lake life
29830
3
gas lamps
29827
1
quartz kitchen counters
29826
4
elegant ambience
29822
1
  awesome
29821
4
austin bungalow
29817
1
metal building
29816
2
hard list
29814
1
large appliances
29813
4
custom builder
29809
3
upgrade fdoor
29806
2
8ft cedar fence
29804
6
double drawer dishwasher
29798
3
pump house
29795
1
new paint in out
29794
2
fixtures 
29792
2
design elements
29790
1
details home
29789
1
beautiful mid century
29788
1
patio wsummer kitchen
29787
3
new boat dock
29784
2
rope lights
29782
1
full kitchen bathroom
29781
3
luxury master suite wcorner tub
29778
9
pecan floors
29769
3
extensive list
29766
1
pot filler faucet
29765
1
wolf gas range
29764
2
exceptional attention detail
29762
3
modern design
29759
2
modern house
29757
2
february completion
29755
2
pebble tec
29753
3
zip line
29750
1

In [ ]:
noun_sentence_df_20.head(10)

In [ ]:
noun_sentence_df_50['AVM_Error_Indicators']

In [ ]:
noun_sentence_df_2000 = noun_sentence_df[noun_sentence_df['count']>2000]

In [ ]:
len(noun_sentence_df_2000)

In [ ]:

import seaborn as sns
sns.set()
def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
    message_embeddings_ = model(messages_)
    plot_similarity(messages_, message_embeddings_, 90)

def plot_similarity_serie(labelref, labels, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = []
    for i in range(len(labels)):
        corr.append(np.inner(message_embeddings_[i,:],messageref_embeddings_))
    
    plt.plot(range(len(labels)),corr)
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from scipy.stats import pearsonr
from gensim.models import Word2Vec

In [ ]:

def plot_similarity_serie_error(labelref, labels, errors, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = np.zeros(len(labels))
    cossim = np.zeros(len(labels))
    worddistance = np.zeros(len(labels))
    
    labelrefwords = labelref[0].split()
    print(labelrefwords)
    
    for i in range(len(labels)):
        labelwords = labels[i].split()
        A = np.array(message_embeddings_[i,:])
        B = np.array(messageref_embeddings_)
        #print(A.shape,B.shape)
        corr[i] = pearsonr(A,B[0])[0]
        #cossim[i] = cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
        
        worddistance[i] = model_gensim.wmdistance(labelrefwords, labelwords)
        
    plt.plot(range(len(labels)),corr)
    plt.plot(range(len(labels)),worddistance)                                 
    
    for i, txt in enumerate(errors):
        plt.annotate(str(round(txt, 3)), xy =((range(len(labels)))[i], corr[i]), ha='center')
        
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr, cossim, worddistance

In [ ]:
run_and_plot(list(noun_sentence_df_1000['noun_sentence'].values[:20]))

In [ ]:

fig = plt.figure(figsize=(15, 6))
plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[0]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
fig = plt.figure(figsize=(17, 6))
corr = plot_similarity_serie_error([noun_sentence_df_1000['noun_sentence'].values[5]],
                                    list(noun_sentence_df_1000['noun_sentence'].values[:40]),
                                    list(noun_sentence_df_1000['AVM_Error_Average'].values[:40]),90)
#print(np.array(corr).shape)
#plt.text(range(len(list(noun_sentence_df_2000['noun_sentence'].values[:40]))), 
#         corr, 
#         str(list(noun_sentence_df_2000['AVM_Error_Average'].values[:40])))

In [ ]:
corr[2]

In [ ]:
fig = plt.figure(figsize=(15, 6))
corr = plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[10]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
fig = plt.figure(figsize=(15, 6))
corr = plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[30]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
np.inner(np.array([0,1]),np.array([0,1]))

In [ ]:
list_of_noun_sentences

In [ ]:
list_of_embeddings

In [ ]:
sentence_possibilities = noun_sentence_df_1000['noun_sentence'].values

In [ ]:
sentence_possibilities

In [ ]:

keys_list = list(noun_sentence_df['noun_sentence'].values)
values_list = [[] for i in range(len(keys_list))]
noun_sentences_dictionary = dict(zip(keys_list, values_list))

In [ ]:
list_of_noun_sentences = []
list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[0])

list_of_embeddings = []
list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[0])

for idd in range(len(noun_sentence_df_1000)):
#for idd in range(50):
    if (idd%30==0): print(idd,len(noun_sentence_df_1000))
    labelrefwords = noun_sentence_df_1000['noun_sentence'].values[idd].split()
    corrs = np.zeros(len(list_of_noun_sentences))
    #distances = np.zeros(len(list_of_noun_sentences))
    #print(corrs)
    for jdd in range(len(list_of_noun_sentences)):
        labelwords = list_of_noun_sentences[jdd].split()
        corrs[jdd] = pearsonr(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[jdd])[0]
        #distances[jdd] = model_gensim.wmdistance(labelrefwords, labelwords)
    #print(corrs)
    #print((corrs<0.79) & (distances>0.5))
    #temp = corrs[((corrs<0.79) & (distances>0.5))]
    temp = corrs[(corrs<0.8)]
    #print(len(temp)==len(list_of_noun_sentences))
                          
    if (len(temp)==len(list_of_noun_sentences)):
        list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[idd])
        list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd])
    else:
        if (len(corrs[(corrs>=0.8) & (corrs!=1)])>0):
            #print(list_of_noun_sentences)
            #print(list((corrs>=0.8) & (corrs!=1)))
            #print(list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)]))
            temp_list = noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]]
            #print(temp_list)
            temp_list.append(noun_sentence_df_1000['noun_sentence'].values[idd])              
            noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]] = temp_list         

In [ ]:
    

for idd in range(len(noun_sentence_df_50)):
#for idd in range(2):
    if (idd%10==0): print(idd,len(noun_sentence_df_50))
        
    count = 0
    corr = 0
    
    noun_sentence_df_50['noun_sentence_embeddings'].values.shape
    
    while ((corr < 0.8) & (corr!=1)):
        #print(noun_sentence_df_50['noun_sentence'].values[idd], ' -- ', list_of_noun_sentences[count])
        corr = pearsonr(noun_sentence_df_50['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[count])[0]
        
        count += 1
        
        #print(corr,count,len(list_of_embeddings))
        
        if (count==len(list_of_embeddings)):
            list_of_noun_sentences.append(noun_sentence_df_50['noun_sentence'].values[idd])
            list_of_embeddings.append(noun_sentence_df_50['noun_sentence_embeddings'].values[idd])
            print(idd,'new', noun_sentence_df_50['noun_sentence'].values[idd])
            break
        
    else:
        print(corr,count,len(list_of_embeddings))
        temp_list = noun_sentences_dictionary[list_of_noun_sentences[count-1]]
        temp_list.append(noun_sentence_df_50['noun_sentence'].values[idd])
        noun_sentences_dictionary[list_of_noun_sentences[count-1]] = temp_list 
    


In [ ]:
noun_sentences_dictionary

In [ ]:
print(noun_sentence_df_50['noun_sentence'].values[idd], ' -- ', list_of_noun_sentences[count-1])


In [ ]:
list_of_noun_sentences

In [ ]:
len(list_of_noun_sentences)

In [ ]:
['rm', 'ss', 'rec', 'fam', 'tlc', 'sep', 'hdwd', 'flrs', 'bd', 'ba', 'fr' ]

In [ ]:
    

for idd in range(len(noun_sentence_df_50)):
#for idd in range(50):
    if (idd%10==0): print(idd,len(noun_sentence_df_50))
        
    labelrefwords = noun_sentence_df_50['noun_sentence'].values[idd].split()
    corrs = np.zeros(len(list_of_noun_sentences))
    #distances = np.zeros(len(list_of_noun_sentences))
    #print(corrs)
    for jdd in range(len(list_of_noun_sentences)):
        labelwords = list_of_noun_sentences[jdd].split()
        corrs[jdd] = pearsonr(noun_sentence_df_50['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[jdd])[0]
        #distances[jdd] = model_gensim.wmdistance(labelrefwords, labelwords)
    #print(corrs)
    #print((corrs<0.79) & (distances>0.5))
    #temp = corrs[((corrs<0.79) & (distances>0.5))]
    temp = corrs[(corrs<0.8)]
    #print(len(temp)==len(list_of_noun_sentences))
                          
    if (len(temp)==len(list_of_noun_sentences)):
        list_of_noun_sentences.append(noun_sentence_df_50['noun_sentence'].values[idd])
        list_of_embeddings.append(noun_sentence_df_50['noun_sentence_embeddings'].values[idd])
    else:
        if (len(corrs[(corrs>=0.8) & (corrs!=1)])>0):
            #print(list_of_noun_sentences)
            #print(list((corrs>=0.8) & (corrs!=1)))
            #print(list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)]))
            temp_list = noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]]
            #print(temp_list)
            temp_list.append(noun_sentence_df_50['noun_sentence'].values[idd])              
            noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]] = temp_list   

In [ ]:
len(list_of_noun_sentences)

In [ ]:
list_of_noun_sentences

In [ ]:
noun_sentence_df_50['noun_sentence'].values[idd]

In [ ]:
noun_sentence_df_50[noun_sentence_df_50['noun_sentence']=='beautiful renovation']

In [ ]:
noun_sentence_df_50['noun_sentence']

In [ ]:
noun_sentence_df_1000[noun_sentence_df_1000['noun_sentence']=='beautiful renovation']

In [ ]:
print(noun_sentence_df_50['noun_sentence'].values[idd])

In [ ]:
list_of_noun_sentences

In [ ]:
list_of_noun_sentences[list_of_noun_sentences=='beautiful renovation']

In [ ]:
list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]

In [ ]:
noun_sentences_dictionary

In [ ]:

keys_list = list(noun_sentence_df_1000['noun_sentence'].values)
values_list = [[] for i in range(len(keys_list))]
noun_sentences_dictionary = dict(zip(keys_list, values_list))

In [ ]:
noun_sentences_dictionary

In [ ]:
len(list_of_noun_sentences)

In [ ]:
        #print(corr_temp, worddistance_tmp)
        if ((corr_temp<0.79) & (worddistance_tmp>0.5)):
            list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[idd])
            list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd])

In [ ]:
list_of_noun_sentences

In [ ]:

def plot_similarity_serie_error(labelref, labels, errors, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = np.zeros(len(labels))
    cossim = np.zeros(len(labels))
    worddistance = np.zeros(len(labels))
    
    labelrefwords = labelref[0].split()
    print(labelrefwords)
    
    for i in range(len(labels)):
        labelwords = labels[i].split()
        A = np.array(message_embeddings_[i,:])
        B = np.array(messageref_embeddings_)
        #print(A.shape,B.shape)
        corr[i] = pearsonr(A,B[0])[0]
        #cossim[i] = cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
        
        worddistance[i] = model_gensim.wmdistance(labelrefwords, labelwords)
        
    plt.plot(range(len(labels)),corr)
    plt.plot(range(len(labels)),worddistance)                                 
    
    for i, txt in enumerate(errors):
        plt.annotate(str(round(txt, 3)), xy =((range(len(labels)))[i], corr[i]), ha='center')
        
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr, cossim, worddistance

In [ ]:

sentence_obama = 'Obama speaks to the media in Illinois'
sentence_president = 'The president greets the press in Chicago'

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

sentence_obama = preprocess(sentence_obama)
sentence_president = preprocess(sentence_president)

In [ ]:
sentence_obama

In [ ]:
sentence_obama.lower().split()

In [ ]:
distance = model_gensim.wmdistance(sentence_obama, sentence_president)
print('distance = %.4f' % distance)

In [ ]:
sentence_orange = preprocess('Oranges are my favorite fruit')
distance = model_gensim.wmdistance(sentence_obama, sentence_orange)
print('distance = %.4f' % distance)

In [ ]:
 model_gensim.wmdistance(['orange'], ['building'])